In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import datetime
import warnings
import numpy as np
import os
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
os.getcwd()


'C:\\Users\\Dhananjay Gupta\\Desktop\\ShaktimanProject5'

In [4]:
shopsdata = pd.read_csv("./shops.csv")
testingdata = pd.read_csv("./test.csv")
salesdata = pd.read_csv("./sales_train_v2.csv")
submissiondata = pd.read_csv("./sample_submission.csv")
itemsdata = pd.read_csv('./items.csv')
item_categoriesdata = pd.read_csv("./item_categories.csv")

In [7]:
traindata = salesdata.join(itemsdata, on='item_id', rsuffix='_').join(shopsdata, on='shop_id', rsuffix='_').join(item_categoriesdata, on='item_category_id', rsuffix='_').drop(['item_id_', 'shop_id_', 'item_category_id_'], axis=1)
traindata.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_name,item_category_id,shop_name,item_category_name
0,02.01.2013,0,59,22154,999.00,1.0,ЯВЛЕНИЕ 2012 (BD),37,"Ярославль ТЦ ""Альтаир""",Кино - Blu-Ray
1,03.01.2013,0,25,2552,899.00,1.0,DEEP PURPLE The House Of Blue Light LP,58,"Москва ТРК ""Атриум""",Музыка - Винил
2,05.01.2013,0,25,2552,899.00,-1.0,DEEP PURPLE The House Of Blue Light LP,58,"Москва ТРК ""Атриум""",Музыка - Винил
3,06.01.2013,0,25,2554,1709.05,1.0,DEEP PURPLE Who Do You Think We Are LP,58,"Москва ТРК ""Атриум""",Музыка - Винил
4,15.01.2013,0,25,2555,1099.00,1.0,DEEP PURPLE 30 Very Best Of 2CD (Фирм.),56,"Москва ТРК ""Атриум""",Музыка - CD фирменного производства


In [8]:
item_ids_test = testingdata['item_id'].unique()
shop_ids_test = testingdata['shop_id'].unique()
train_lk = traindata[traindata['shop_id'].isin(shop_ids_test)]
train_lk = train_lk[train_lk['item_id'].isin(item_ids_test)]

In [74]:
traindata = traindata.query('item_price > 0')
traindata.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_name,item_category_id,shop_name,item_category_name
0,02.01.2013,0,59,22154,999.00,1.0,ЯВЛЕНИЕ 2012 (BD),37,"Ярославль ТЦ ""Альтаир""",Кино - Blu-Ray
1,03.01.2013,0,25,2552,899.00,1.0,DEEP PURPLE The House Of Blue Light LP,58,"Москва ТРК ""Атриум""",Музыка - Винил
2,05.01.2013,0,25,2552,899.00,-1.0,DEEP PURPLE The House Of Blue Light LP,58,"Москва ТРК ""Атриум""",Музыка - Винил
3,06.01.2013,0,25,2554,1709.05,1.0,DEEP PURPLE Who Do You Think We Are LP,58,"Москва ТРК ""Атриум""",Музыка - Винил
4,15.01.2013,0,25,2555,1099.00,1.0,DEEP PURPLE 30 Very Best Of 2CD (Фирм.),56,"Москва ТРК ""Атриум""",Музыка - CD фирменного производства


In [75]:
monthly_train = train_lk[['date', 'date_block_num', 'shop_id', 'item_category_id', 'item_id', 'item_price', 'item_cnt_day']]

In [76]:
monthly_train = monthly_train.sort_values('date').groupby(['date_block_num', 'shop_id', 'item_category_id', 'item_id'], as_index=False)
monthly_train = monthly_train.agg({'item_price':['sum', 'mean'], 'item_cnt_day':['sum', 'mean','count']})
monthly_train.columns = ['date_block_num', 'shop_id', 'item_category_id', 'item_id', 'item_price', 'mean_item_price', 'item_cnt', 'mean_item_cnt', 'transactions']
monthly_train

,date_block_num,shop_id,item_category_id,item_id,item_price,mean_item_price,item_cnt,mean_item_cnt,transactions
0,0,2,2,5572,10730.00,1532.857143,9.0,1.285714,7
1,0,2,2,5643,4775.21,2387.605000,0.0,0.000000,2
2,0,2,5,5583,1188.30,594.150000,2.0,1.000000,2
3,0,2,6,7893,5970.00,1990.000000,3.0,1.000000,3
4,0,2,6,7894,1490.00,1490.000000,1.0,1.000000,1
5,0,2,6,7895,2697.00,899.000000,4.0,1.333333,3
6,0,2,6,7956,13780.00,6890.000000,2.0,1.000000,2
7,0,2,19,1409,1398.50,1398.500000,1.0,1.000000,1
8,0,2,19,1467,899.00,899.000000,1.0,1.000000,1
9,0,2,19,3076,1399.00,1399.000000,1.0,1.000000,1


In [77]:
shop_ids = monthly_train['shop_id'].unique()
item_ids = monthly_train['item_id'].unique()
empty_df = []
for i in range(34):
    for shop in shop_ids:
        for item in item_ids:
            empty_df.append([i, shop, item])
    
empty_df = pd.DataFrame(empty_df, columns=['date_block_num','shop_id','item_id'])

In [78]:
monthly_train = pd.merge(empty_df, monthly_train, on=['date_block_num','shop_id','item_id'], how='left')
monthly_train.fillna(0, inplace=True)
monthly_train

,date_block_num,shop_id,item_id,item_category_id,item_price,mean_item_price,item_cnt,mean_item_cnt,transactions
0,0,2,5572,2.0,10730.00,1532.857143,9.0,1.285714,7.0
1,0,2,5643,2.0,4775.21,2387.605000,0.0,0.000000,2.0
2,0,2,5583,5.0,1188.30,594.150000,2.0,1.000000,2.0
3,0,2,7893,6.0,5970.00,1990.000000,3.0,1.000000,3.0
4,0,2,7894,6.0,1490.00,1490.000000,1.0,1.000000,1.0
5,0,2,7895,6.0,2697.00,899.000000,4.0,1.333333,3.0
6,0,2,7956,6.0,13780.00,6890.000000,2.0,1.000000,2.0
7,0,2,1409,19.0,1398.50,1398.500000,1.0,1.000000,1.0
8,0,2,1467,19.0,899.00,899.000000,1.0,1.000000,1.0
9,0,2,3076,19.0,1399.00,1399.000000,1.0,1.000000,1.0


In [79]:
monthly_train['year'] = monthly_train['date_block_num'].apply(lambda x: ((x//12) + 2013))
monthly_train['month'] = monthly_train['date_block_num'].apply(lambda x: (x % 12))

In [80]:
monthly_train = monthly_train.query('item_cnt >= 0 and item_cnt <= 20 and item_price < 400000')
monthly_train['item_cnt_month'] = monthly_train.sort_values('date_block_num').groupby(['shop_id', 'item_id'])['item_cnt'].shift(-1)

C:\Python\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [81]:
monthly_train['item_price_unit'] = monthly_train['item_price'] // monthly_train['item_cnt']
monthly_train['item_price_unit'].fillna(0, inplace=True)

C:\Python\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Python\Anaconda\lib\site-packages\pandas\core\generic.py:4355: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [82]:
item_price_gp = monthly_train.sort_values('date_block_num').groupby(['item_id'], as_index=False).agg({'item_price':[np.min, np.max]})
item_price_gp.columns = ['item_id', 'hist_min_item_price', 'hist_max_item_price']

monthly_train = pd.merge(monthly_train, item_price_gp, on='item_id', how='left')

In [83]:
monthly_train['price_increase'] = monthly_train['item_price'] - monthly_train['hist_min_item_price']
monthly_train['price_decrease'] = monthly_train['hist_max_item_price'] - monthly_train['item_price']
monthly_train

,date_block_num,shop_id,item_id,item_category_id,item_price,mean_item_price,item_cnt,mean_item_cnt,transactions,year,month,item_cnt_month,item_price_unit,hist_min_item_price,hist_max_item_price,price_increase,price_decrease
0,0,2,5572,2.0,10730.00,1532.857143,9.0,1.285714,7.0,2013,0,1.0,1192.0,0.0,18979.500000,10730.00,8249.500000
1,0,2,5643,2.0,4775.21,2387.605000,0.0,0.000000,2.0,2013,0,0.0,0.0,0.0,35260.000000,4775.21,30484.790000
2,0,2,5583,5.0,1188.30,594.150000,2.0,1.000000,2.0,2013,0,1.0,594.0,0.0,5592.000000,1188.30,4403.700000
3,0,2,7893,6.0,5970.00,1990.000000,3.0,1.000000,3.0,2013,0,2.0,1990.0,0.0,27950.000000,5970.00,21980.000000
4,0,2,7894,6.0,1490.00,1490.000000,1.0,1.000000,1.0,2013,0,2.0,1490.0,0.0,25880.000000,1490.00,24390.000000
5,0,2,7895,6.0,2697.00,899.000000,4.0,1.333333,3.0,2013,0,1.0,674.0,0.0,8447.000000,2697.00,5750.000000
6,0,2,7956,6.0,13780.00,6890.000000,2.0,1.000000,2.0,2013,0,0.0,6890.0,0.0,35230.000000,13780.00,21450.000000
7,0,2,1409,19.0,1398.50,1398.500000,1.0,1.000000,1.0,2013,0,0.0,1398.0,0.0,5596.000000,1398.50,4197.500000
8,0,2,1467,19.0,899.00,899.000000,1.0,1.000000,1.0,2013,0,0.0,899.0,0.0,5994.000000,899.00,5095.000000
9,0,2,3076,19.0,1399.00,1399.000000,1.0,1.000000,1.0,2013,0,3.0,1399.0,0.0,12559.765000,1399.00,11160.765000


In [9]:
# Mean value
mean_f = lambda x: x.rolling(window=3, min_periods=1).mean()
# Max value
maximum_f = lambda x: x.rolling(window=3, min_periods=1).max()
minimum_f = lambda x: x.rolling(window=3, min_periods=1).min()
# Standard deviation
std_f = lambda x: x.rolling(window=3, min_periods=1).std()

name_fun = ['min', 'max', 'mean', 'std']
list_fun = [minimum_f, maximum_f, mean_f, std_f]


for i in range(len(list_fun)):
    monthly_train[('item_cnt_%s' % name_fun[i])] = monthly_train.sort_values('date_block_num').groupby(['shop_id', 'item_category_id', 'item_id'])['item_cnt'].apply(list_fun[i])

# Fill the empty std features with 0
monthly_train['item_cnt_std'].fillna(0, inplace=True)

NameError: name 'maximum_f' is not defined

In [85]:
list_lag = [1, 2, 3]

for lag in list_lag:
    name_ft = ('item_cnt_shifted%s' % lag)
    monthly_train[name_ft] = monthly_train.sort_values('date_block_num').groupby(['shop_id', 'item_category_id', 'item_id'])['item_cnt'].shift(lag)
    monthly_train[name_ft].fillna(0, inplace=True)

In [86]:
monthly_train['item_trend'] = monthly_train['item_cnt']

for lag in list_lag:
    name_ft = ('item_cnt_shifted%s' % lag)
    monthly_train['item_trend'] -= monthly_train[name_ft]

monthly_train['item_trend'] /= len(list_lag) + 1
monthly_train

,date_block_num,shop_id,item_id,item_category_id,item_price,mean_item_price,item_cnt,mean_item_cnt,transactions,year,...,price_increase,price_decrease,item_cnt_min,item_cnt_max,item_cnt_mean,item_cnt_std,item_cnt_shifted1,item_cnt_shifted2,item_cnt_shifted3,item_trend
0,0,2,5572,2.0,10730.00,1532.857143,9.0,1.285714,7.0,2013,...,10730.00,8249.500000,9.0,9.0,9.0,0.0,0.0,0.0,0.0,2.25
1,0,2,5643,2.0,4775.21,2387.605000,0.0,0.000000,2.0,2013,...,4775.21,30484.790000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00
2,0,2,5583,5.0,1188.30,594.150000,2.0,1.000000,2.0,2013,...,1188.30,4403.700000,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.50
3,0,2,7893,6.0,5970.00,1990.000000,3.0,1.000000,3.0,2013,...,5970.00,21980.000000,3.0,3.0,3.0,0.0,0.0,0.0,0.0,0.75
4,0,2,7894,6.0,1490.00,1490.000000,1.0,1.000000,1.0,2013,...,1490.00,24390.000000,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.25
5,0,2,7895,6.0,2697.00,899.000000,4.0,1.333333,3.0,2013,...,2697.00,5750.000000,4.0,4.0,4.0,0.0,0.0,0.0,0.0,1.00
6,0,2,7956,6.0,13780.00,6890.000000,2.0,1.000000,2.0,2013,...,13780.00,21450.000000,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.50
7,0,2,1409,19.0,1398.50,1398.500000,1.0,1.000000,1.0,2013,...,1398.50,4197.500000,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.25
8,0,2,1467,19.0,899.00,899.000000,1.0,1.000000,1.0,2013,...,899.00,5095.000000,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.25
9,0,2,3076,19.0,1399.00,1399.000000,1.0,1.000000,1.0,2013,...,1399.00,11160.765000,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.25


In [87]:
set_train = monthly_train.query('date_block_num >= 3 and date_block_num < 28').copy()
set_valid = monthly_train.query('date_block_num >= 28 and date_block_num < 33').copy()
set_test = monthly_train.query('date_block_num == 33').copy()

In [88]:
mean_gp_shop = monthly_train.groupby(['shop_id']).agg({'item_cnt_month': ['mean']})
mean_gp_shop.columns = ['shop_mean']
mean_gp_shop.reset_index(inplace=True)

In [89]:
set_train.dropna(subset=['item_cnt_month'], inplace=True)
set_valid.dropna(subset=['item_cnt_month'], inplace=True)

In [90]:
set_train.dropna(inplace=True)
set_valid.dropna(inplace=True)

In [91]:
mean_gp_item = monthly_train.groupby(['item_id']).agg({'item_cnt_month': ['mean']})
mean_gp_item.columns = ['item_mean']
mean_gp_item.reset_index(inplace=True)

In [92]:
gp_shop_item_mean = monthly_train.groupby(['shop_id', 'item_id']).agg({'item_cnt_month': ['mean']})
gp_shop_item_mean.columns = ['shop_item_mean']
gp_shop_item_mean.reset_index(inplace=True)

In [93]:
gp_year_mean = monthly_train.groupby(['year']).agg({'item_cnt_month': ['mean']})
gp_year_mean.columns = ['year_mean']
gp_year_mean.reset_index(inplace=True)

In [94]:
mean_gp_month = monthly_train.groupby(['month']).agg({'item_cnt_month': ['mean']})
mean_gp_month.columns = ['month_mean']
mean_gp_month.reset_index(inplace=True)

In [95]:
set_train = pd.merge(monthly_train, mean_gp_shop, on=['shop_id'], how='left')
set_train = pd.merge(set_train, mean_gp_item, on=['item_id'], how='left')
set_train = pd.merge(set_train, gp_shop_item_mean, on=['shop_id', 'item_id'], how='left')
set_train = pd.merge(set_train, gp_year_mean, on=['year'], how='left')
set_train = pd.merge(set_train, mean_gp_month, on=['month'], how='left')

In [96]:
set_valid = pd.merge(set_valid, mean_gp_shop, on=['shop_id'], how='left')
set_valid = pd.merge(set_valid, mean_gp_item, on=['item_id'], how='left')
set_valid = pd.merge(set_valid, gp_shop_item_mean, on=['shop_id', 'item_id'], how='left')
set_valid = pd.merge(set_valid, gp_year_mean, on=['year'], how='left')
set_valid = pd.merge(set_valid, mean_gp_month, on=['month'], how='left')

In [99]:
train_X = set_train.drop(['item_cnt_month', 'date_block_num'], axis=1)
Y_train = set_train['item_cnt_month']
X_validation = set_valid.drop(['item_cnt_month', 'date_block_num'], axis=1)
Y_validation = set_valid['item_cnt_month'].astype(int)
print(X_validation)
print(Y_validation)

        shop_id  item_id  item_category_id  item_price  mean_item_price  \
0             2     5572               2.0      1590.0          1590.00   
1             2     5643               0.0         0.0             0.00   
2             2     5583               0.0         0.0             0.00   
3             2     7893               0.0         0.0             0.00   
4             2     7894               6.0      4579.5          2289.75   
5             2     7895               0.0         0.0             0.00   
6             2     7956               0.0         0.0             0.00   
7             2     1409               0.0         0.0             0.00   
8             2     1467               0.0         0.0             0.00   
9             2     3076              19.0      2398.0          1199.00   
10            2     3316               0.0         0.0             0.00   
11            2     3686               0.0         0.0             0.00   
12            2     3851 

In [100]:
int_features = ['shop_id', 'item_id', 'year', 'month']
train_X[int_features] = train_X[int_features].astype('int32')
X_validation[int_features] = X_validation[int_features].astype('int32')

In [101]:
latest_records = pd.concat([set_train, set_valid]).drop_duplicates(subset=['shop_id', 'item_id'], keep='last')
X_test = pd.merge(testingdata, latest_records, on=['shop_id', 'item_id'], how='left', suffixes=['', '_'])
X_test['year'] = 2015
X_test['month'] = 9
X_test.drop('item_cnt_month', axis=1, inplace=True)
X_test[int_features] = X_test[int_features].astype('int32')
X_test = X_test[train_X.columns]
X_test.head()

,shop_id,item_id,item_category_id,item_price,mean_item_price,item_cnt,mean_item_cnt,transactions,year,month,...,item_cnt_std,item_cnt_shifted1,item_cnt_shifted2,item_cnt_shifted3,item_trend,shop_mean,item_mean,shop_item_mean,year_mean,month_mean
0,5,5037,19.0,749.5,749.5,1.0,1.0,1.0,2015,9,...,1.154701,3.0,1.0,1.0,-1.00,0.137392,0.806941,0.393939,0.236645,0.196448
1,5,5320,NaN,NaN,NaN,NaN,NaN,NaN,2015,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5,5233,19.0,2997.0,999.0,3.0,1.0,3.0,2015,9,...,1.000000,1.0,2.0,3.0,-0.75,0.137392,0.352092,0.303030,0.236645,0.196448
3,5,5232,0.0,0.0,0.0,0.0,0.0,0.0,2015,9,...,0.000000,0.0,0.0,0.0,0.00,0.137392,0.101010,0.030303,0.236645,0.196448
4,5,5268,NaN,NaN,NaN,NaN,NaN,NaN,2015,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [115]:
sets = [train_X, X_validation, X_test]
for dataset in sets:
    for shop_id in dataset['shop_id'].unique():
        for column in dataset.columns:
            shop_median = dataset[(dataset['shop_id'] == shop_id)][column].median()
            dataset.loc[(dataset[column].isnull()) & (dataset['shop_id'] == shop_id), column] = shop_median
            
X_test.fillna(X_test.mean(), inplace=True)

In [114]:
train_X.drop(['item_category_id'], axis=1, inplace=True)
X_validation.drop(['item_category_id'], axis=1, inplace=True)
X_test.drop(['item_category_id'], axis=1, inplace=True)
X_test.describe().T

ValueError: labels ['item_category_id'] not contained in axis

In [107]:
lr_features = ['item_cnt', 'item_cnt_shifted1', 'item_trend', 'mean_item_cnt', 'shop_mean']
lr_train = train_X[lr_features]
lr_val = X_validation[lr_features]
lr_test = X_test[lr_features]

In [108]:
lr_scaler = MinMaxScaler()
lr_scaler.fit(lr_train)
lr_train = lr_scaler.transform(lr_train)
lr_val = lr_scaler.transform(lr_val)
lr_test = lr_scaler.transform(lr_test)

In [113]:
lr_train_pred = lr_model.predict(lr_train)
lr_val_pred = lr_model.predict(lr_val)
lr_test_pred = lr_model.predict(lr_test)

NotFittedError: This LinearRegression instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.

In [ ]:
print('Train rmse:', np.sqrt(mean_squared_error(Y_train, lr_train_pred)))
print('Validation rmse:', np.sqrt(mean_squared_error(Y_validation, lr_val_pred)))

In [ ]:
first_level['linear_regression'] = lr_val_pred
first_level.head()

In [112]:
first_level_test['linear_regression'] = lr_test_pred
first_level_test.tail()

NameError: name 'lr_test_pred' is not defined

In [ ]:
meta_model = LinearRegression(n_jobs=-1)
meta_model

In [ ]:
first_level.drop('label', axis=1, inplace=True)
meta_model.fit(first_level, Y_validation)

In [ ]:
ensemble_pred = meta_model.predict(first_level)
final_predictions = meta_model.predict(first_level_test)

In [ ]:
print('Train rmse:', np.sqrt(mean_squared_error(ensemble_pred, Y_validation)))

In [106]:
prediction_df = pd.DataFrame(testingdata['ID'], columns=['ID'])
prediction_df['item_cnt_month'] = final_predictions.clip(0., 20.)
prediction_df.to_csv('submission.csv', index=False)
prediction_df.head()

NameError: name 'final_predictions' is not defined